# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

Create some Forex contracts:

In [2]:
contracts = [Forex(pair) for pair in 'EURUSD USDJPY GBPUSD USDCHF USDCAD AUDUSD'.split()]

eurusd = contracts[0]

Request streaming ticks for them:

In [3]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [4]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', exchange='IDEALPRO'), time=datetime.datetime(2018, 5, 8, 13, 6, 30, 746835, tzinfo=datetime.timezone.utc), bid=1.18552, bidSize=11500000, ask=1.18554, askSize=2000000, prevBid=1.18554, prevBidSize=10500000, prevAsk=1.18556, prevAskSize=7400000, high=1.19385, low=1.1855, close=1.1923, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[])

The price of Forex ticks is always nan. To get a midpoint price use ``marketPrice()``.

The tickers are kept
live updated, try this a few times to see if the price changes:

In [5]:
ticker.marketPrice()

1.18553

The following cell will start a 60 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [7]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(columns='symbol bidSize bid ask askSize high low close'.split())
df['symbol'] = [c.symbol + c.currency for c in contracts]
contract2Row = {c: i for (i, c) in enumerate(contracts)}

def onPendingTickers(tickers):
    for t in tickers:
        row = contract2Row[t.contract]
        df.iloc[row, 1:] = (t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers

ib.sleep(60)

,symbol,bidSize,bid,ask,askSize,high,low,close
0,EURUSD,4000000,1.1856,1.18563,5500000,1.19385,1.18515,1.1923
1,USDJPY,1000000,109.222,109.223,2000000,109.24,108.855,109.09
2,GBPUSD,2000000,1.3497,1.34971,2000000,1.35935,1.34915,1.3557
3,USDCHF,2000000,1.00269,1.00271,4000000,1.0045,1.0016,1.0027
4,USDCAD,2000000,1.29722,1.29724,1000000,1.2985,1.2876,1.2881
5,AUDUSD,2000000,0.74426,0.74427,1500000,0.7528,0.7442,0.7518


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [8]:
for contract in contracts:
    ib.cancelMktData(contract)

In [9]:
ib.disconnect()